In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/My Drive/Machine Learning/food_classifer_dataset.zip" -d "/content/"

Archive:  /content/drive/My Drive/Machine Learning/food_classifer_dataset.zip
replace /content/classifer_dataset/train/Pizza/pizza1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/classifer_dataset/train/Pizza/pizza10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# this is going to help in data processing
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#these libraries are going to help in the model building.
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense

In [ ]:

# dimensions of our images.
img_width, img_height = 150, 150

#setting up the directories
train_data_dir = '/content/classifer_dataset/train'
validation_data_dir = '/content/classifer_dataset/validate'

#setting up the batchsizes.
nb_train_samples = 8400
nb_validation_samples = 1600
#below are the hyerparameters
epochs = 50
batch_size = 16

In [ ]:
input_shape = (img_width, img_height, 3)

In [ ]:
# this is the augmentation configuration we will use for training
# augumentation generates more training images by rescaling, shearing, etc
train_datagen = ImageDataGenerator(
    rescale=1./ 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)



In [ ]:
#this generates batches of augment data for training
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 8400 images belonging to 2 classes.


In [ ]:
# this is the augmentation configuration we will use for validating
val_datagen = ImageDataGenerator(rescale=1./255)

#this generates batches of augment data for validating
validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 1600 images belonging to 2 classes.


In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
#for extracting more features, I am adding more number of convolutional layers.
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#flatten converts the image matrix to a 1-D vector.
model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#below dense function manages the output neuron.
model.add(Dense(1, activation='sigmoid'))


In [ ]:
#configuring the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#to print a summary representation of your model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
batch_normalization_4 (Batch (None, 148, 148, 32)      128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
batch_normalization_5 (Batch (None, 72, 72, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 64)       

In [ ]:
#model training
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/50
525/525 [==============================] - 96s 183ms/step - loss: 0.3969 - accuracy: 0.8436 - val_loss: 0.3918 - val_accuracy: 0.8406
Epoch 2/50
525/525 [==============================] - 95s 182ms/step - loss: 0.2480 - accuracy: 0.9025 - val_loss: 0.4171 - val_accuracy: 0.8600
Epoch 3/50
525/525 [==============================] - 95s 181ms/step - loss: 0.2198 - accuracy: 0.9146 - val_loss: 0.1885 - val_accuracy: 0.9212
Epoch 4/50
525/525 [==============================] - 95s 180ms/step - loss: 0.2051 - accuracy: 0.9233 - val_loss: 0.2640 - val_accuracy: 0.8981
Epoch 5/50
525/525 [==============================] - 94s 180ms/step - loss: 0.1956 - accuracy: 0.9196 - val_loss: 0.2021 - val_accuracy: 0.9169
Epoch 6/50
525/525 [==============================] - 95s 181ms/step - loss: 0.1746 - accuracy: 0.9313 - val_loss: 0.1879 - val_accuracy: 0.9337
Epoch 7/50
525/525 [==============================] - 96s 182ms/step - loss: 0.1833 - accuracy: 0.9304 - val_loss: 0.1799 - val_ac

In [ ]:
#to save the weights in the model as a HDFS file

model.save_weights('model_weight.h5')

#to save the architecture of the model as a json file

with open('model_architecture.json','w') as f:
    f.write(model.to_json())

In [ ]:
from keras.models import load_model
from keras.models import model_from_json

In [ ]:
# Model reconstruction from JSON file
with open('/content/food_model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('/content/first_try.h5')

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image=image.load_img('/content/images (1).jfif',target_size=(img_width,img_height))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=model.predict(test_image)
print(result)

In [ ]:
if result[0][0]==1.0:
    prediction='SoftDrink'
else:
    prediction='Pizza'
print("You got a "+ prediction + " Yupeeeeeeeeeeeeeeeeeeeeeeeeeeeeee!!!")